In [1]:
import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu0')

Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 3007)


In [2]:
import random

import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [3]:
import numpy as np
np.random.seed(1985)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [4]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 8
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

In [5]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

('X_train shape:', (60000, 1, 28, 28))
(60000, 'train samples')
(10000, 'test samples')


In [6]:
def init_model():
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            input_shape=(1, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adadelta')
    return model

In [7]:
def evalModel(individual):
    score = individual[0].evaluate(X_test, Y_test, show_accuracy=True, verbose=0)[1]
    return score,

In [22]:
def cloneModel(individual):
    return individual

In [31]:
def cxTwoPointCopy(ind1, ind2):
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]
        
    return ind1, ind2

In [32]:
def crossoverModels(ind1, ind2):
    w1 = ind1[0].get_weights()
    w2 = ind2[0].get_weights()
    
    for i in range(len(w1)):
        if len(w1[i].shape) == 4:
            w1, w2 = cxTwoPointCopy(w1, w2)
            
    ind1[0].set_weights(w1)
    ind2[0].set_weights(w2)
    return ind1, ind2

In [39]:
def mutateModel(individual, epochs):
    individual[0].fit(X_train, Y_train, batch_size=batch_size, nb_epoch=epochs,
          show_accuracy=True, verbose=0, validation_data=(X_test, Y_test))
    return individual,    

In [40]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [41]:
toolbox = base.Toolbox()

toolbox.register("init_model", init_model)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.init_model, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [42]:
toolbox.register("evaluate", evalModel)
toolbox.register("mate", crossoverModels)
toolbox.register("mutate", mutateModel, epochs=1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("clone", cloneModel)

In [43]:
def main():
    random.seed(64)
    
    pop = toolbox.population(n=3)
    
    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    # hof = tools.HallOfFame(1, similar=np.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=2.0, ngen=40, stats=stats) #, halloffame=hof)

    
    return pop, stats, hof

In [44]:
pop, stats, hof = main()

gen	nevals	avg     	std      	min   	max   
0  	3     	0.120433	0.0169266	0.0978	0.1385
1  	3     	0.9677  	0.00494975	0.9607	0.9712
2  	3     	0.9776  	1.11022e-16	0.9776	0.9776
3  	3     	0.9804  	0          	0.9804	0.9804
4  	3     	0.9824  	0          	0.9824	0.9824
5  	3     	0.9819  	0          	0.9819	0.9819
6  	3     	0.9799  	1.11022e-16	0.9799	0.9799
7  	3     	0.9822  	1.11022e-16	0.9822	0.9822
8  	3     	0.9816  	1.11022e-16	0.9816	0.9816
9  	3     	0.9814  	1.11022e-16	0.9814	0.9814
10 	3     	0.9801  	1.11022e-16	0.9801	0.9801
11 	3     	0.9831  	0          	0.9831	0.9831
12 	3     	0.9833  	0          	0.9833	0.9833
13 	3     	0.9832  	0          	0.9832	0.9832
14 	3     	0.9841  	1.11022e-16	0.9841	0.9841
15 	3     	0.984   	0          	0.984 	0.984 
16 	3     	0.9815  	0          	0.9815	0.9815
17 	3     	0.9838  	0          	0.9838	0.9838
18 	3     	0.982   	1.11022e-16	0.982 	0.982 
19 	3     	0.984   	0          	0.984 	0.984 


KeyboardInterrupt: 